In [1]:
# Importing the required libraries 
import nltk
import numpy as np
import itertools

In [2]:
# Downloading and importing the brown corpus
nltk.download('brown')
from nltk.corpus import brown

[nltk_data] Downloading package brown to
[nltk_data]     C:\Users\lenovo\AppData\Roaming\nltk_data...
[nltk_data]   Package brown is already up-to-date!


In [12]:
#Getting the tagged sentences
sent_tag = brown.tagged_sents()
brown_sent_tag=[]
for s in sent_tag:
    s.insert(0,('##','##'))
    s.append(('&&','&&'))
    brown_sent_tag.append(s)

In [13]:
#Splitting the data for train and test
split_num = int(len(brown_sent_tag)*0.9)
train_data = brown_sent_tag[0:split_num]
test_data = brown_sent_tag[split_num:]

In [14]:
# MAking a dict() variable to store TAG , word and its count in the following fashion:
# {'TAG' : {word : 'count of word with TAG'}  }

train_word_tag = {}
for data in train_data:
    #print(data)
    for word, tag in data:
        #print(word,tag)
        #print('------')
        word = word.lower()
        if tag in train_word_tag:
            if word in train_word_tag[tag]:
                train_word_tag[tag][word] +=1
            else:
                train_word_tag[tag][word]=1
        else:
            train_word_tag[tag] ={word:1}

In [16]:
# Making the emission probability matrix

# Making a nested disctionary to in the following fashion:
# { Tag : {word: emission probability form that tag' s state} }

# This is the variable to store the dictionaty
train_emission_prob_matrix = {}
for tag in train_word_tag.keys():
    
    # creating the blank nested dictionary for {word : emission prob}
    train_emission_prob_matrix[tag] = {}
    
    # calculating number of word with that tag
    count = sum(train_word_tag[tag].values())
    
    for word in train_word_tag[tag].keys():
        
        # Calculating the emmission prob of each word for that state or Tag
        train_emission_prob_matrix[tag][word] = train_word_tag[tag][word]/count
train_emission_prob_matrix

{'##': {'##': 1.0},
 'AT': {'the': 0.7097017949251476,
  'an': 0.037847842037825936,
  'no': 0.018058764907515908,
  'a': 0.22949634772704872,
  'every': 0.004884299059279621,
  "th'": 1.0951343182241303e-05},
 'NP-TL': {'fulton': 0.0025886616619207868,
  'atlanta': 0.0010354646647683149,
  'grady': 0.0010354646647683149,
  'georgia': 0.0036241263266891016,
  'jackson': 0.0005177323323841574,
  'miller': 0.0005177323323841574,
  'colquitt': 0.0002588661661920787,
  'texas': 0.004400724825265338,
  'dallas': 0.003365260160497023,
  'beaumont': 0.0002588661661920787,
  'lamar': 0.0005177323323841574,
  'texan': 0.0002588661661920787,
  'wise': 0.0002588661661920787,
  'paris': 0.000776598498576236,
  'oklahoma': 0.000776598498576236,
  'rhode': 0.02562775045301579,
  'massachusetts': 0.0036241263266891016,
  'u.s.': 0.014496505306756407,
  'denton': 0.0002588661661920787,
  'york': 0.07377685736474243,
  'st.': 0.0028475278281128655,
  'louis': 0.003365260160497023,
  'cook': 0.000517732